In [ ]:
# default_exp models.tacotron2.prenet

In [ ]:
# export

import pdb
from torch import nn
import numpy as np
import torch
from torch.autograd import Variable
from torch.cuda.amp import autocast
from torch.nn import functional as F
from uberduck_ml_dev.models.common import LinearNorm
import math


class Dummy(nn.Module):
    """
        Dummy function for None activation
    """

    def __init__(self):
        super(Dummy, self).__init__()

    def forward(self, x):
        return x


## Activation Function used in Other Module
ACTIVATION_GROUP = {"linear": Dummy, "tanh": nn.Tanh, "relu": nn.ReLU}

# class Prenet(nn.Module):
#     def __init__(self, in_dim, sizes):
#         super().__init__()
#         in_sizes = [in_dim] + sizes[:-1]
#         self.layers = nn.ModuleList(
#             [
#                 LinearNorm(in_size, out_size, bias=False)
#                 for (in_size, out_size) in zip(in_sizes, sizes)
#             ]
#         )
#         self.dropout_rate = 0.5

#     def forward(self, x):
#         for linear in self.layers:
#             x = F.dropout(F.relu(linear(x)), p=self.dropout_rate, training=True)
#         return x

# NOTE (Sam): non-attentive taco version
class Prenet(nn.Module):
    def __init__(self, in_dim, sizes, dropout_p=0.5, activation="relu"):
        super(Prenet, self).__init__()

        assert (
            activation in ACTIVATION_GROUP
        ), "activation must either one of them [{}]".format(
            ", ".join(ACTIVATION_GROUP.keys())
        )

        in_sizes = [in_dim] + sizes[:-1]
        self.layers = nn.ModuleList(
            [
                LinearNorm(in_size, out_size, bias=False)
                for (in_size, out_size) in zip(in_sizes, sizes)
            ]
        )

        self.dropout = nn.Dropout(p=dropout_p)
        self.activation = ACTIVATION_GROUP[activation]()

    def forward(self, x):
        for linear in self.layers:
            x = self.dropout(self.activation(linear(x)))
        return x